# Guardado y Carga de Modelos
En esta sección vamos a aprender como se realiza el guardado y carga de modelos.

## Configuración


In [ ]:
import tensorflow as tf

## Guardar modelos secuenciales o modelos funcionales

Consideremos el siguiente modelo:

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Opcionalmente, entrenaremos este modelo, solo para que tenga valores de peso para guardarlos, así como un estado optimizador. Por supuesto, tambien puede guardar modelos que nunca ha entrenado, pero obviamente eso es menos interesante.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

In [ ]:
# Guardar predicciones para futuras verificaciones
predictions = model.predict(x_test)

## Guardar Modelo-Completo

Podemos guardar un modelo creado con la API funcional en un solo archivo. Posteriormente, podemos volver a crear el mismo modelo a partir de este archivo, incluso si ya no tenemos acceso al codigo que creamos el modelo.

Este archivo incluye:

- Los modelos de arquitectura
- Los valores de peso del modelo (que se aprendieron durante el entrenamiento)
- La configuración de entrenamiento del modelo (lo que pasó a 'compilar'), si corresponde
- El optimizador y su estado, si corresponde (esto le permite reiniciar el entrenamiento donde lo dejamos)

In [ ]:
# Guardar el Modelo
model.save('path_to_my_model.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('path_to_my_model.h5')

import numpy as np

# Verifique que el estado esté preservado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Tenga en cuenta que el estado del optimizador también se conserva:
# puede reanudar el entrenamiento donde lo dejó.

## Exportar a 'SavedModel'
Tambien podemos exportar un modelo completo al formato 'SavedModel' de TensorFlow. 'SavedModel' es un formato de serialización independiente para objetos de Tensorflow, compatible con el servicio de TensorFlow y las implementaciones de TensorFlow que no sean Python.

In [ ]:
# Exportar el modelo a 'SavedModel'
keras.experimental.export_saved_model(model, 'path_to_saved_model')

# Recrea exactamente el mismo modelo
new_model = keras.experimental.load_from_saved_model('path_to_saved_model')

# Verifica que el estado esté guardado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Tenemos que tener cuenta que el estado del optimizador también se conserva:
# puede reanudar el entrenamiento donde lo dejamos.

Los archivos 'SavedModel' que se crearon contienen:

- Un punto de control TensorFlow que contiene los pesos del modelo.
- Un prototipo 'SavedModel' que contiene el grafico subyacente de Tensorflow. Separar los graficos que se guardan para prediccion (servicio), capacitacion y evaluacion. Si el modelo no se compilo antes, solo el grafico de inferencia se exporta
- La configuracion de arquitectura del modelo, si esta disponible.

## Solo arquitectura de guardado
A veces, solo estamos interesadso en la arquitectura del modelo y no necesitamos guardar los valores de peso o el optimizador. En este caso, podemos recuperar la "configuracion" del modelo mediante el metodo get_config(). La configuracion es un diccionario de Python que nos permite recrear el mismo modelo, inicializado desde cero, sin ninguna de la información aprendida previamente durante el entrenamiento.

In [ ]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

# ¡OJO: el estado del modelo no se conserva! Solo guardamos la arquitectura.
new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

## Guardando solo con pesos
A veces, solo nos interesa el estado del modelo, sus valores de peso, y no la arquitectura. En este caso, podemos recuperar los valores de pesos como una lista de matrices Numpy a traves de 'get_weights()', y establecer el estado del modelo a través de 'set_weights':

In [ ]:
weights = model.get_weights()  # Recupera el estado del modelo.
model.set_weights(weights)  # Establece el estado del modelo.

Podemos combinar get_config()/from_config() y get_weights()/set_weights() para recrear nuestro modelo en el mismo estado. Sin embargo, a diferencia de model.save(), esto no incluira la configuracion de entrenamiento y el optimizado tendría que volver a llamar a compile() antes de usar el modelo para el entrenamiento.

In [ ]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

# Verifica que el estado esté preservado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# OJO: el optimizador no se conserva
# entonces el modelo debe compilarse nuevamente antes de entrenar
# (y el optimizador comenzará desde un estado en blanco).

**Pero recuerda que la forma más simple y recomendada es solo esto:**

In [ ]:
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')